In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# Create Model class that inherits nn.Module
class Model(nn.Module):
  # Input layer(4 features of the flower) --> Hidden layer 1(number of neurons) --> Hidden layer 2(number of neurons) --> Output layer(3 classes)
  def __init__(self, in_features=4, h1=8, h2=8, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1) # fc - full connected
    self.fc2 = nn.Linear(h1, h2)
    self.out_put = nn.Linear(h2, out_features)
  
  def forward(self, x):
    x = F.relu(self.fc1(x)) # relu - rectified linear unit
    x = F.relu(self.fc2(x))
    x = self.out_put(x)
    return x

In [5]:
# Pick a manual seed for randomization

torch.manual_seed(41)

# Create a model instance

model = Model()

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
my_df = pd.read_csv(url)
my_df


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica
